In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install pythainlp

In [ ]:
import pandas as pd
import numpy as np
from pythainlp.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction import stop_words
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import re
import itertools


#For Neural Network
from keras import models
from keras.models import Sequential
from keras import layers
from keras.preprocessing.sequence import pad_sequences
#from keras.utils.vis_utils import plot_model
from keras.utils import pad_sequences

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/sentiment_lagalce.csv")
print(data.head(20))

       class                                           sentence
0   positive             เบบี้บลัชใช้ดีมาก ทาแล้วแก้มชมพูน่ารัก
1   positive  เราชอบมาก น่ารักมาก ๆ เลยค่ะ หมดเมื่อไหร่กลับม...
2   positive                                  เลิศเกิน สีสวยมาก
3   positive  เห็นแพคเกจในคลิปรีวิวว่ากรี๊ดแล้ว เห็นของจริงย...
4   positive  เอาจริง ๆ น้องใช้โคตรดี ดีแบบอึ้ง ว่าแล้วทำไมด...
5   positive  แบรนด์นี้มาก ฮิตมาก เนื้อสัมผัสดี สีละมุนมาก ก...
6   positive  แพ็คเก็จน่ารัก มินิมอลดี เนื้อคอนซิลเลอร์เนียน...
7   positive  แพคเกจสวยมาก ๆ ปริมาณเพิ่มขึ้น ราคาเท่าเดิม ใช...
8   positive  แพคเกจสวยมากชอบมาก หลังจากลองใช้ค่อนข้างปกปิดด...
9   positive                             ใช้ดีมาก ซื้อซ้ำแน่นอน
10  positive                                   ใช้ดีมาก สวยจึ้ง
11  positive          กลิตเตอร์น่ารักมาก ๆ ค่ะ ประกายวิ้งวับมาก
12  positive  กลิตเตอร์สวยจึ้งมาก เป็นกลิตเตอร์ที่แน่นสวยมากค่ะ
13  positive                 คอนทัวร์แบรนด์นี้ติดดีสุด ๆ เลยค่ะ
14  positive                            

In [ ]:
sentiment_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}

In [ ]:
data['class'] = data['class'].replace(sentiment_mapping)

In [ ]:
print(data.head(20))

    class                                           sentence
0       2             เบบี้บลัชใช้ดีมาก ทาแล้วแก้มชมพูน่ารัก
1       2  เราชอบมาก น่ารักมาก ๆ เลยค่ะ หมดเมื่อไหร่กลับม...
2       2                                  เลิศเกิน สีสวยมาก
3       2  เห็นแพคเกจในคลิปรีวิวว่ากรี๊ดแล้ว เห็นของจริงย...
4       2  เอาจริง ๆ น้องใช้โคตรดี ดีแบบอึ้ง ว่าแล้วทำไมด...
5       2  แบรนด์นี้มาก ฮิตมาก เนื้อสัมผัสดี สีละมุนมาก ก...
6       2  แพ็คเก็จน่ารัก มินิมอลดี เนื้อคอนซิลเลอร์เนียน...
7       2  แพคเกจสวยมาก ๆ ปริมาณเพิ่มขึ้น ราคาเท่าเดิม ใช...
8       2  แพคเกจสวยมากชอบมาก หลังจากลองใช้ค่อนข้างปกปิดด...
9       2                             ใช้ดีมาก ซื้อซ้ำแน่นอน
10      2                                   ใช้ดีมาก สวยจึ้ง
11      2          กลิตเตอร์น่ารักมาก ๆ ค่ะ ประกายวิ้งวับมาก
12      2  กลิตเตอร์สวยจึ้งมาก เป็นกลิตเตอร์ที่แน่นสวยมากค่ะ
13      2                 คอนทัวร์แบรนด์นี้ติดดีสุด ๆ เลยค่ะ
14      2                                  คุณภาพคุ้มราคามาก
15      2  ชอบแบรนด์ลากล

In [ ]:
sentence = data['sentence'].astype(str)
y = data['class'].values

In [ ]:
X_tokenized = sentence.apply(word_tokenize, keep_whitespace=False)
print(X_tokenized)

0      [เบบี้, บลัช, ใช้, ดีมาก, ทา, แล้ว, แก้ม, ชมพู...
1      [เรา, ชอบ, มาก, น่ารัก, มาก, ๆ, เลย, ค่ะ, หมด,...
2                             [เลิศ, เกิน, สี, สวย, มาก]
3      [เห็น, แพคเกจ, ใน, คลิป, รีวิว, ว่า, กรี๊ด, แล...
4      [เอา, จริง ๆ, น้อง, ใช้, โคตร, ดี, ดี, แบบ, อึ...
                             ...                        
146    [สินค้า, ไม่, ติด, ทน, เหมือน, ที่แจ้ง, ไว้, ส...
147    [สินค้า, ของ, ข้างใน, ดี, ทุกอย่าง, ยกเว้น, ขน...
148              [สี, ไม่, เหมือน, ที่, รีวิว, เท่าไหร่]
149               [ห่วย, มาก, ทาง, ร้าน, บริการ, แย่มาก]
150    [อาย, ไลเนอร์, แห้ง, เพิ่ง, ใช้, ไม่, กี่, ครั...
Name: sentence, Length: 151, dtype: object


In [ ]:
#train test spit
sentence_train, sentence_test, y_train, y_test = train_test_split(X_tokenized, y, test_size=0.20, random_state=100)

In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence_train)
tokenizer.word_index

In [ ]:
X_train = tokenizer.texts_to_sequences(sentence_train)
X_test = tokenizer.texts_to_sequences(sentence_test)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

444


In [ ]:
print(sentence_train[0])

['เบบี้', 'บลัช', 'ใช้', 'ดีมาก', 'ทา', 'แล้ว', 'แก้ม', 'ชมพู', 'น่ารัก']


In [ ]:
print(X_train[0])

[58, 67, 35, 83, 3, 59, 109, 68]


In [ ]:
#ทำข้อความเท่ากัน ละเช็คความยาวจร๊
from keras.utils import to_categorical
print("Raw Texts to Sequence : ")
for s in X_train: print(s)
maxlen = max([len(s) for s in X_train])
print("Max sentence length = ",maxlen)
x = pad_sequences(X_train, maxlen=maxlen, padding="post")
print("Sequence Padding at maxlen words, post padding : \n",x)

Raw Texts to Sequence : 
[58, 67, 35, 83, 3, 59, 109, 68]
[16, 18, 60, 69, 27, 145, 28, 8]
[45, 46, 47, 7, 110, 1, 27, 45, 46, 47, 13, 146, 7, 1, 4]
[32, 19, 236, 1, 147, 148, 1]
[26, 19, 69, 36, 149, 61, 1, 84, 3, 7, 1, 237]
[3, 7, 1, 48, 28]
[150, 151, 23, 32, 238, 5, 84, 20, 152, 62]
[17, 153, 16, 18, 60, 2, 11, 24, 6, 70, 85, 239, 86, 240, 1, 241, 4]
[2, 111, 154, 5, 14, 39, 242, 155, 68, 12, 243, 156, 244, 245, 63, 40, 112, 157, 49, 4]
[37, 87, 64, 1, 113, 158, 37, 40, 114, 6, 29, 1, 8, 88, 71, 115, 40, 116, 159, 246, 29, 1, 8, 89, 21, 7, 1, 8, 247, 29, 39, 117, 248, 5]
[160, 62, 118, 50, 16, 18, 60, 23, 24, 41, 84, 119, 14, 249, 120, 86, 114, 68, 21, 6, 4]
[38, 250, 90, 91, 51, 92, 33, 161, 20, 6, 93, 42, 40, 20, 121, 162, 251, 8, 4, 2, 11, 122, 2, 42, 163]
[2, 11, 27, 65, 13, 90, 164, 118, 50, 252, 7, 165, 34, 52, 253, 34, 23, 166, 41]
[7, 3, 7, 1, 167, 28, 32, 1]
[72, 168, 29, 254, 255, 256, 6, 26, 16, 18, 73, 257, 258, 24, 6, 87, 123, 1]
[169, 29, 1, 72, 56, 259, 28, 8]
[170, 

In [ ]:
#Word Embedding
from keras.models import Sequential
from keras.layers import Embedding , Dense ,Flatten
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

embedding_dim = 128
model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    input_length=maxlen,
                    output_dim=embedding_dim
                           ))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 54, 128)           56832     
                                                                 
 flatten_18 (Flatten)        (None, 6912)              0         
                                                                 
 dense_39 (Dense)            (None, 128)               884864    
                                                                 
 dense_40 (Dense)            (None, 1)                 129       
                                                                 
Total params: 941825 (3.59 MB)
Trainable params: 941825 (3.59 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(x,y_train,epochs=10)

Epoch 1/10
4/4 [==============================] - 1s 21ms/step - loss: 0.0000e+00 - accuracy: 0.3417
Epoch 2/10
4/4 [==============================] - 0s 17ms/step - loss: 0.0000e+00 - accuracy: 0.3583
Epoch 3/10
4/4 [==============================] - 0s 16ms/step - loss: 0.0000e+00 - accuracy: 0.4417
Epoch 4/10
4/4 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 0.4917
Epoch 5/10
4/4 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.4833
Epoch 6/10
4/4 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.5000
Epoch 7/10
4/4 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 0.5250
Epoch 8/10
4/4 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 0.5250
Epoch 9/10
4/4 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.5167
Epoch 10/10
4/4 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accura

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,Bidirectional,Dense,LSTM
embedding_dim = 128
model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    input_length=maxlen,
                    output_dim=embedding_dim
                           ))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 54, 128)           56832     
                                                                 
 bidirectional (Bidirection  (None, 256)               263168    
 al)                                                             
                                                                 
 dense_24 (Dense)            (None, 1)                 257       
                                                                 
Total params: 320257 (1.22 MB)
Trainable params: 320257 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(x,y_train,
          epochs=20)

Epoch 1/20
4/4 [==============================] - 0s 51ms/step - loss: 0.0000e+00 - accuracy: 0.5250
Epoch 2/20
4/4 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - accuracy: 0.5250
Epoch 3/20
4/4 [==============================] - 0s 69ms/step - loss: 0.0000e+00 - accuracy: 0.5167
Epoch 4/20
4/4 [==============================] - 0s 64ms/step - loss: 0.0000e+00 - accuracy: 0.5250
Epoch 5/20
4/4 [==============================] - 0s 50ms/step - loss: 0.0000e+00 - accuracy: 0.5250
Epoch 6/20
4/4 [==============================] - 0s 57ms/step - loss: 0.0000e+00 - accuracy: 0.5333
Epoch 7/20
4/4 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - accuracy: 0.5250
Epoch 8/20
4/4 [==============================] - 0s 36ms/step - loss: 0.0000e+00 - accuracy: 0.5250
Epoch 9/20
4/4 [==============================] - 0s 51ms/step - loss: 0.0000e+00 - accuracy: 0.5250
Epoch 10/20
4/4 [==============================] - 0s 34ms/step - loss: 0.0000e+00 - accura